# LangChain - Multiple retrievers

## Install dependencies

In [1]:
%pip install langchain
%pip install openai
%pip install chromadb
%pip install tiktoken
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\cglyn\appdata\local\programs\python\python311\lib\site-packages\requests-2.31.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\cglyn\appdata\local\programs\python\python311\lib\site-packages\websockets-11.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://gi

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\cglyn\appdata\local\programs\python\python311\lib\site-packages\requests-2.31.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\cglyn\appdata\local\programs\python\python311\lib\site-packages\websockets-11.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://gi

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\cglyn\appdata\local\programs\python\python311\lib\site-packages\requests-2.31.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\cglyn\appdata\local\programs\python\python311\lib\site-packages\websockets-11.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://gi

DEPRECATION: Loading egg at c:\users\cglyn\appdata\local\programs\python\python311\lib\site-packages\requests-2.31.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\cglyn\appdata\local\programs\python\python311\lib\site-packages\websockets-11.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://gi

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\cglyn\appdata\local\programs\python\python311\lib\site-packages\requests-2.31.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\cglyn\appdata\local\programs\python\python311\lib\site-packages\websockets-11.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://gi

## Load environment variables

Change the path if necessary (read `README.md` first and follow the instructions to setup the `.env` file):

In [ ]:
# import os
# os.chdir("/home/jovyan/work/")

In [2]:
%load_ext dotenv
%dotenv

Disable warnings:

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Load a repository

In [4]:
from langchain.document_loaders.text import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import urllib.request

In [18]:
embeddings = OpenAIEmbeddings(disallowed_special=())

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 10810,
    chunk_overlap  = 200,
    length_function = len,
)

In [19]:
db = Chroma(embedding_function=embeddings)

In [20]:
from langchain.vectorstores.base import VectorStoreRetriever


def ingest_book(book_url: str, book_name: str, db: Chroma) -> VectorStoreRetriever:
    file_path = f"/tmp/{book_name}.txt"
    urllib.request.urlretrieve(book_url, file_path)

    loader = TextLoader(file_path)
    documents = loader.load()

    texts = text_splitter.split_documents(documents)

    chunks = [doc.page_content for doc in texts]
    metadatas = [doc.metadata for doc in texts]
    db.add_texts(texts=chunks, metadatas=metadatas)

    return db.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 6,
            "filter": {
                "source": file_path,
            }
        },
    )

In [21]:
retriever1 = ingest_book("https://www.gutenberg.org/files/55317/55317-0.txt", "Meditations", db)

In [38]:
import requests

def ingest_book(url, title, db):
    response = requests.get(url)
    if response.status_code == 200:
        text = response.content.decode("utf-8")
        # Process the text further (e.g., store it in the database)
    else:
        print("Failed to retrieve the book from the URL")

# Example usage
retriever2 = ingest_book("https://www.gutenberg.org/ebooks/12096.txt.utf-8", "Bushido", db)


## Router

In [36]:
from langchain.chains.router import MultiRetrievalQAChain
from langchain.chat_models import ChatOpenAI

**Note:** change between models, or tweak splitter and retrievers parameters, if you have issues. It works well with GPT-4.

In [37]:
llm = ChatOpenAI(temperature=0)
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
#llm = ChatOpenAI(model="gpt-4", temperature=0)

In [39]:
retriever_infos = [
    {
        "name": "meditations",
        "description": "Good for answering questions about Meditations by Marcus Aurelius",
        "retriever": retriever1,
    },
    {
        "name": "bushido",
        "description": "Good for answering questions about Bushido book",
        "retriever": retriever2,
    },
]

In [41]:
qa = MultiRetrievalQAChain.from_retrievers(llm, retriever_infos)

ValidationError: 1 validation error for RetrievalQA
retriever
  none is not an allowed value (type=type_error.none.not_allowed)

## Test router

In [42]:
qa("How the Bushido approachs the concept of personal honor?")

{'input': 'How the Bushido approachs the concept of personal honor?',
 'query': 'How does Bushido approach the concept of personal honor?',
 'result': "Bushido, the code of moral principles followed by the samurai in feudal Japan, places a significant emphasis on personal honor. The concept of personal honor in Bushido is closely tied to loyalty, courage, integrity, and self-discipline. Samurai were expected to act with honor in all aspects of their lives, to uphold their word, and to maintain their dignity even in the face of adversity. Dishonorable behavior was considered a grave offense, often leading to severe consequences. The idea of personal honor in Bushido is deeply rooted in the belief that one's actions reflect not only on oneself but also on one's family, ancestors, and descendants."}

In [43]:
qa("How Marcus Aurelio approachs the concept of personal honor?")

{'input': 'How Marcus Aurelio approachs the concept of personal honor?',
 'query': 'How Marcus Aurelius approaches the concept of personal honor?',
 'result': "Marcus Aurelius approaches the concept of personal honor by emphasizing the importance of staying true to one's principles and duties, regardless of external circumstances. He believes that a person should focus on living a life in accordance with their nature and the principles of good and evil. He advises against seeking reputation among others and instead encourages individuals to act in a way that aligns with virtues like justice, temperance, courage, and freedom. Aurelius suggests that true honor comes from fulfilling one's responsibilities and living a life guided by moral principles, rather than seeking external validation or recognition."}

## Agent

In [29]:
from langchain.agents import initialize_agent, Tool

In [30]:
tools = [
    Tool(
        name = "Meditations",
        func=retriever1.get_relevant_documents,
        description="Useful for answering questions about Meditations by Marcus Aurelius."
    ),
    # Tool(
    #     name = "Bushido",
    #     func=retriever1.get_relevant_documents,
    #     description="Useful for answering questions about Bushido book."
    # ),
]

In [31]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

c:\Users\cglyn\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


## Test agent

In [32]:
print(agent.run("What is the perspective of Marcus Aurelius and Bushido on the nature of suffering and adversity, and how do they recommend dealing with these challenges?"))

c:\Users\cglyn\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
It would be helpful to consult Meditations by Marcus Aurelius to understand his perspective on suffering and adversity.
Action: [Meditations]
Action Input: Read Book 4 of Meditations
Observation: [Meditations] is not a valid tool, try one of [Meditations].
Thought:I should try using the tool "Meditations" instead of [Meditations].
Action: [Meditations]
Action Input: Read Book 4 of Meditations
Observation: [Meditations] is not a valid tool, try one of [Meditations].
Thought:I should use the tool "Meditations" to find the perspective of Marcus Aurelius on suffering and adversity.
Action: [Meditations]
Action Input: Read Book 4 of Meditations
Observation: [Meditations] is not a valid tool, try one of [Meditations].
Thought:I should try using the tool "Meditations" to understand Marcus Aurelius' perspective on suffering and adversity.
Action: [Meditations]
Action Input: Read Book 4 of Meditations
Observation: [Meditations] is not a valid tool, try on

In [ ]:
print(agent.run("How do Marcus Aurelius and the principles of Bushido approach the pursuit of inner peace and tranquility in the midst of external chaos and conflict?"))

In [44]:
print(agent.run("In what ways do Marcus Aurelius and the principles of Bushido emphasize the importance of constant self-improvement and lifelong learning?"))



> Entering new AgentExecutor chain...
Both Marcus Aurelius and the principles of Bushido emphasize the importance of constant self-improvement and lifelong learning through their teachings and practices.
Action: Meditations
Action Input: Marcus Aurelius' writings on self-improvement and learning
Observation: [Document(page_content="ï»¿The Project Gutenberg eBook of The Meditations of the Emperor Marcus Aurelius Antoninus, by Marcus Aurelius\n\nThis eBook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this eBook or online at\nwww.gutenberg.org. If you are not located in the United States, you\nwill have to check the laws of the country where you are located before\nusing this eBook.\n\nTitle: The Meditations of the Emperor Marcus Aurelius Antoninus\nA new rendering based on the Foul

In [ ]:
print(agent.run("What are the differences between how Marcus Aurelius and the Bushido code address the role of emotions in decision-making and daily life?"))